In [1]:
import sys,os
sys.path.append('../../')
from smodels.base import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'
import numpy as np
from smodels.decomposition import decomposer
from smodels.base.physicsUnits import fb, GeV, TeV
from smodels.matching.theoryPrediction import theoryPredictionsFor,TheoryPredictionsCombiner
from smodels.experiment.databaseObj import Database
from smodels.base.smodelsLogging import setLogLevel
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.tools.coverage import Uncovered,UncoveredGroup
setLogLevel("error")
%load_ext line_profiler
# Set the path to the database
database = Database('unittest')

### Load model

In [2]:
slhafile = '../../inputFiles/slha/lightEWinos.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 1*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               massCompress=True, invisibleCompress=True,
                               minmassgap=mingap)

In [4]:
print(len(topDict.getSMSList()))
totWeight = None
for el in topDict.getSMSList():
    if totWeight is None:
        totWeight = el.weightList.copy()
    else:
        totWeight += el.weightList
print(totWeight.getMaxXsec())

917
1.40E+01 [pb]


### Get all results

In [5]:
listOfExpRes = database.getExpResults()

### Compute the theory predictions

In [6]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = theoryPredictionsFor(database, topDict, combinedResults=False)
predsDict = {}
for tp in allPredictions:
    anaID = tp.analysisId()
    if anaID not in predsDict:
        predsDict[anaID] = []
    predsDict[anaID].append(tp)

# Print results sorted by analysis ID
for anaID in sorted(predsDict.keys()):
    tpList = predsDict[anaID]
    if not tpList:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % anaID)
    for theoryPrediction in tpList:
        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)


 Theory Predictions and Constraints:

 ATLAS-CONF-2013-037 
r = 1.142E-01

 ATLAS-SUSY-2013-02 
r = 6.560E-05

 ATLAS-SUSY-2013-12 
r = 6.460E-01

 CMS-SUS-13-012 
r = 5.050E-03


In [7]:
# import time
# t0 = time.time()
uncovered = Uncovered(topDict, sqrts=13.*TeV)
# print(time.time()-t0)

In [8]:
# First sort coverage groups by label
groups = sorted(uncovered.groups[:], key=lambda g: g.label)
# Print uncovered cross-sections:
for group in groups:
    print("\nTotal cross-section for %s (fb): %10.3E\n" % (group.description, group.getTotalXSec()))


Total cross-section for missing topologies (fb):  1.046E+04


Total cross-section for missing topologies with displaced decays (fb):  0.000E+00


Total cross-section for missing topologies with prompt decays (fb):  1.046E+04


Total cross-section for topologies outside the grid (fb):  8.055E+01



In [9]:
missingTopos = uncovered.getGroup('missing (prompt)')
# Print some of the missing topologies:
finalStates = {}
for iel,genEl in enumerate(missingTopos.finalStateSMS[:]):
    elStr = str(genEl)
    if elStr not in finalStates:
        finalStates[elStr] = genEl.missingX
    else:
        finalStates[elStr] += genEl.missingX

            # Print some of the missing topologies:
print(len(finalStates))
for fs in sorted(finalStates.keys()):
    print('Element:', fs)
    print('\tcross-section (fb):', np.round(finalStates[fs],4))        

166
Element: PV > (MET), (MET)
	cross-section (fb): 3.8833
Element: PV > (MET), (W,jet,jet,MET)
	cross-section (fb): 5.076
Element: PV > (MET), (Z,MET)
	cross-section (fb): 1.8942
Element: PV > (MET), (b,b,MET)
	cross-section (fb): 1.1665
Element: PV > (MET), (jet,jet,MET)
	cross-section (fb): 352.4218
Element: PV > (MET), (nu,l,MET)
	cross-section (fb): 109.4426
Element: PV > (MET), (nu,ta,MET)
	cross-section (fb): 54.5767
Element: PV > (W,b,t,MET), (b,b,b,b,MET)
	cross-section (fb): 8.1348
Element: PV > (W,b,t,MET), (b,b,jet,jet,MET)
	cross-section (fb): 30.0312
Element: PV > (W,b,t,MET), (b,t,jet,jet,MET)
	cross-section (fb): 36.5879
Element: PV > (W,jet,jet,MET), (Z,jet,jet,MET)
	cross-section (fb): 8.0204
Element: PV > (W,jet,jet,jet,MET), (W,jet,jet,jet,MET)
	cross-section (fb): 130.8067
Element: PV > (W,jet,jet,jet,MET), (Z,b,t,jet,jet,MET)
	cross-section (fb): 34.8003
Element: PV > (W,jet,jet,jet,MET), (b,b,b,b,MET)
	cross-section (fb): 88.9474
Element: PV > (W,jet,jet,jet,MET)

In [10]:
missingDisplaced = uncovered.getGroup('missing (displaced)')
# Print elements with displaced decays:
if missingDisplaced.finalStateSMS:
    for genEl in missingDisplaced.finalStateSMS[:3]:
        print('Element:', genEl)
        print('\tcross-section (fb):', np.round(genEl.missingX,4))
else:
    print("\nNo displaced decays")



No displaced decays


In [11]:
missingGrid = uncovered.getGroup('outsideGrid (all)')
finalStates = {}
for iel,genEl in enumerate(missingGrid.finalStateSMS[:]):
    elStr = str(genEl)
    if elStr not in finalStates:
        finalStates[elStr] = genEl.missingX
    else:
        finalStates[elStr] += genEl.missingX

print(len(finalStates))
for fs in sorted(finalStates.keys()):
    print('Element:', fs)
    print('\tcross-section (fb):', np.round(finalStates[fs],4))        

2
Element: PV > (b,b,MET), (b,b,MET)
	cross-section (fb): 72.7112
Element: PV > (t,t,MET), (b,b,MET)
	cross-section (fb): 7.8367
